# Putting It All Together - The 8s Puzzle Part 2

## The Game

<img src = "files/8sPuzzle1.png" width = "200">

## Representing the Game

This is the class we developed to represent the 8s puzzle game in part 1.

In [ ]:
import random

# A class to represent the 8s puzzle sliding tile game
class Puzzle8():

    # A private class variable storing the characeter we use as the blank
    # Done this way to avoid a hard coded string being repeated in the code, 
    # this could lead to errors as we might get it wrong sometimes
    blankChar = " "
    
    # Create a new game board - either from a set of positions, or by defaul A - D
    def __init__(self, tiles = None):
        # Creat the tile board as a list - note that the 
        if tiles == None:
            self.tiles = [["A", "B", "C"], ["D", Puzzle8.blankChar, "E"], ["F", "G", "H"]]
        else:
            self.tiles = tiles
    
        # Note the position of the blank, as it has probably moved
        self.blankPos = self.getBlankPos()
        
        # Error checking - if no blank character, put one in
        if self.blankPos == [-1, -1]:
            self.tiles[0][0] = Puzzle8.blankChar
            self.blankPos = [0, 0]
            
    # Shuffle the game board
    def shuffle(self, numMoves = 5):
        
        # perform a large number of random moves
        for i in range(0, numMoves):
            move = random.choice(["left", "right", "up", "down"])
            self.moveBlank(move)
            
        # Note the position of the blank, as it has probably moved
        self.blankPos = self.getBlankPos()
      
    # Print the game board
    def printTiles(self, short = False):
        
        if short == True:
            print(str(self.tiles[0]) + " | " + str(self.tiles[1]) + " | " + str(self.tiles[2]))
        else:
            print("-------------")
            print("| " + self.tiles[0][0] + " | " + self.tiles[0][1] + " | " + self.tiles[0][2]  + " |")
            print("-------------")
            print("| " + self.tiles[1][0] + " | " + self.tiles[1][1] + " | " + self.tiles[1][2]  + " |")
            print("-------------")
            print("| " + self.tiles[2][0] + " | " + self.tiles[2][1] + " | " + self.tiles[2][2]  + " |")
            print("-------------")

        
    # Find the position of the blank space - returns a list 
    # containing the row and column indices in that order
    def getBlankPos(self):
        row = -1
        col = -1
        for row in range(0, 3):
            if Puzzle8.blankChar in self.tiles[row]:
                col = self.tiles[row].index(Puzzle8.blankChar)
                break
        return [row, col]
            
    def moveBlank(self, direction):
        
        # Make direction lowercase and trimmed
        direction = direction.lower()
        direction = direction.strip()
        
        # Find where the blank is - not really needed but just in case it has gone out of date!
        self.blankPos = self.getBlankPos()
        
        result = False
        
        # Check what direction is required and move appropriately
        # (use starts with so we can pass "left" or "l" etc)
        if direction.startswith("l"):
            result = self.moveBlankLeft()
        elif direction.startswith("r"):
            result = self.moveBlankRight()
        elif direction.startswith("u"):
            result = self.moveBlankUp()
        elif direction.startswith("d"):
            result = self.moveBlankDown()
        return result
        
    # Move the blank space one place to the left
    def moveBlankLeft(self):
        
        # Check that the blank is not already at the left limit. If it is leave it there and return false
        if(self.blankPos[1] == 0):
            return False
        else:
            self.tiles[self.blankPos[0]][self.blankPos[1]] = self.tiles[self.blankPos[0]][self.blankPos[1] - 1]
            self.tiles[self.blankPos[0]][self.blankPos[1] - 1] = Puzzle8.blankChar
            return True
            
    # Move the blank space one place to the right
    def moveBlankRight(self):
        
        # Check that the blank is not already at the right limit. If it is leave it there and return false
        if(self.blankPos[1] == 2):
            return False
        else:
            self.tiles[self.blankPos[0]][self.blankPos[1]] = self.tiles[self.blankPos[0]][self.blankPos[1] + 1]
            self.tiles[self.blankPos[0]][self.blankPos[1] + 1] = Puzzle8.blankChar
            return True
        
    # Move the blank space one place up
    def moveBlankUp(self):
        
        # Check that the blank is not already at the top limit. If it is leave it there and return false
        if(self.blankPos[0] == 0):
            return False
        else:
            self.tiles[self.blankPos[0]][self.blankPos[1]] = self.tiles[self.blankPos[0] - 1][self.blankPos[1]]
            self.tiles[self.blankPos[0] - 1][self.blankPos[1]] = Puzzle8.blankChar
            return True
        
    # Move the blank space one place down
    def moveBlankDown(self):
        
        # Check that the blank is not already at the bottom limit. If it is leave it there and return false
        if(self.blankPos[0] == 2):
            return False
        else:
            self.tiles[self.blankPos[0]][self.blankPos[1]] = self.tiles[self.blankPos[0] + 1][self.blankPos[1]]
            self.tiles[self.blankPos[0] + 1][self.blankPos[1]] = Puzzle8.blankChar
            return True
        
    # check if the current state of the board mathces a template
    def matchTemplate(self, template):
        
        # Check that the temaplte is the same size as the game board
        if len(template) == 0 or len(template) != len(self.tiles) or len(template[0]) != len(self.tiles[0]):
            return False
       
        # If the sizes match iterate through the eloements in the boards caomparing each one. 
        # Assume from the start that they match, then if we get to the end without finding a mismatch they are the same
        # If we find any mismatch bail out
        else:
            
            match = True
            for r in range(0, len(template)):
                for c in range(0, len(template[r])):

                    # If there is a mismatch get out
                    if self.tiles[r][c] != template[r][c]:
                        match = False
                        break
            
            return match
    
    # Overload == and != opertors so we can easily search lists for boards
    def __eq__(self, other):
        return self.matchTemplate(other.tiles)
    def __ne__(self, other):
        return not self.matchTemplate(other.tiles)

## Searching for Solutions

We can think about each possible board configuration as a **state**. We can think about the different moves we can make (left, right, up, down) as moving us between different states. We can en build a big tree of states and move through it. 

In [ ]:
import copy

class SearchState:
    
    def __init__(self, board, parent = None, moveFromParent = ""):
        # Store links tot he parent of this state and the move that got from there to here
        self.parent = parent
        self.moveFromParent = moveFromParent
        
        # Make a deep copy of the board
        self.board = copy.deepcopy(board)
        
        # Make an empty list for children
        self.children = list()

        # Store how far down the search tree this search state is
        if parent is not None:
            self.treeDepth = parent.treeDepth + 1
        else:
            self.treeDepth = 0
    
    # Generate the list of the child states of this node
    def createChildren(self):
        
        # Iterate through all possible moves trying to kae a child
        for m in ['left', 'right', 'up', 'down']:
            # Create a copy of the current board and perofrm the move in it
            childBoard = copy.deepcopy(self.board)
            legalMove = childBoard.moveBlank(m)
            
            # As long as the move was allowed then add it to the child list
            if legalMove == True:
                childState = SearchState(childBoard, self, m)
                self.children.append(childState)

    # Print the search state     
    def show(self, short = False, showChildren = True, showParent= True):
        self.board.printTiles(short)
        
        # If there are children then print them too
        if showChildren and len(self.children) > 0:
            print("CHILDREN")
            for c in self.children:
                print(c.moveFromParent)
                c.show(short, False, False)
        
        # If there is one print the parent
        if showParent and self.parent is not None:
            print("PARENT")
            self.parent.show(short, False, False)
                
    # Overload == and != opertors so we can easily search lists for boards
    def __eq__(self, other):
        return self.board == other.board
    def __ne__(self, other):
        return self.board != other.board

Show what a search state can tell us

In [ ]:
start = Puzzle8()
start.shuffle()

ss = SearchState(start)
ss.createChildren()

ss.show()

Test the equality operators in the SearchState class

In [ ]:
start = Puzzle8()
ssStart = SearchState(start)

goal = Puzzle8()
ssGoal = SearchState(goal)

ssStart.show()
ssGoal.show()

if(ssStart == ssGoal):
    print("States are the same")
else:
    print("States are different")
    
goal = Puzzle8()
goal.shuffle()
ssGoal = SearchState(goal)

ssStart.show()
ssGoal.show()

if(ssStart == ssGoal):
    print("States are the same")
else:
    print("States are different")

Implement a **breadth first search**.

In [ ]:
from collections import deque # Needed for the queue

# A class to perform a breadth first search
class BreadthFirstSearch:
    
    # Constructor
    def __init__(self):
        self.root = None
        self.queue = deque()
    
    # Perform a search from a start state to a goal state
    def search(self, start, goal):
        
        # Count the number of states searched
        count = 0
        
        # Create a search state object from the starting state
        root = start
        
        # Put the starting state on top of the stack
        self.queue.append(root)
        
        # While there are more states on the stack keep seearching
        while len(self.queue) > 0:
            
            # Pop the latest state off the stack
            currentState = self.queue.popleft()
            
            # Check if the current state is the goal - if so we are done!
            if currentState == goal:
                
                # Print some details
                print("FOUND!!!!")
                print("States searched " + str(count))
                currentState.show(True, False, False)
                print("QUEUE " + str(len(self.queue)))
                
                # Iterate back through the parents to find the move sequence to get here
                state = currentState
                moveList = list()
                while state.parent is not None:
                    moveList.append(state.moveFromParent)
                    state = state.parent                
                moveList.reverse() # Reverse the list of moves and print to get the sequence to win!
                return moveList # Return the list of moves that corresponds to the solution
            
            # if the current state has children add those that are not on the queue to the queue
            currentState.createChildren()
            for c in currentState.children:
                
                # Check if the current child state has already been considered on the way to this point in the tree
                # Move back up the tree using the parent links and compare each parent with the current state
                movingState = currentState
                currentStateAlreadyOnPath = False
                while movingState.parent is not None:
                    movingState = movingState.parent
                    if(movingState == currentState):
                        currentStateAlreadyOnPath = True
                        break
                
                # Only put the current state onto the stack if it isn't already on the path here
                if currentStateAlreadyOnPath == False:
                    self.queue.append(c)
                
            # Increment the number of states searched
            count += 1   
                
            # Every thousand states seaerched print out the current state of affairs
            if(count % 1000 == 0):
                print("------------------")
                currentState.show(True, False, False)
                print("States searched " + str(count))
                print("Tree depth: " + str(currentState.treeDepth))
                print("QUEUE " + str(len(self.queue)))

        # If we get to here it means we haven't ben able to find a solution so return None
        return None  
 

Test out the search.

In [ ]:
searcher = BreadthFirstSearch()

# Try out a search
startBoard = Puzzle8()
startBoard.shuffle(100)
start = SearchState(startBoard)
goal = SearchState(Puzzle8())
#start = SearchState(Puzzle8())
#goal = SearchState(Puzzle8([["D", "A", "C"], ["F", "B", Puzzle8.blankChar], ["G", "H", "E"]]))

print("Start")
start.show()
print("Goal")
goal.show()

moves = searcher.search(start, goal)

# Print the path to a solution
if moves is not None:
    start.board.printTiles()
    count = 1
    for m in moves:
        print(" ")
        print("Move " + str(count) + " " + m)
        start.board.moveBlank(m)
        start.board.printTiles()
        count += 1

Implement a **depth first search**.

In [ ]:
# A class to perform a depth first search
class DepthFirstSearch:
    
    # Constructor
    def __init__(self):
        self.root = None
        self.stack = list()
    
    # Perform a search from teh start state to the goal state
    def search(self, start, goal):
        
        # Count the number of states searched
        count = 0
        
        # Create a search state object from the starting state
        root = start
        
        # Put the starting state on top of the stack
        self.stack.append(root)
        
        # While there are more states on the stack keep searching
        while len(self.stack) > 0:
            
            # Pop the latest state off the stack
            currentState = self.stack.pop()

            # check if the current state is the goal
            if currentState == goal:

                # Print details fo tehg goal!
                print("FOUND!!!!")
                print("States searched " + str(count))
                currentState.show(True, False, False)
                print("STACK " + str(len(self.stack)))
                
                # Iterate back through the parents to find the move sequence to get here
                state = currentState
                moveList = list()
                while state.parent is not None:
                    moveList.append(state.moveFromParent)
                    state = state.parent
                moveList.reverse()# Reverse the list of moves and print to get the sequence to win!                
                return moveList # Returnt he list of moves to go from start to goal
            
            # if the current state has children add those that are not on either the visitied list or the stack to the stack
            currentState.createChildren()
            for c in currentState.children:

                # Check if the current child state has already been considered on the way to this point in the tree
                # Move back up the tree using the parent links and compare each parent with the current state
                movingState = currentState
                currentStateAlreadyOnPath = False
                while movingState.parent is not None:
                    movingState = movingState.parent
                    if(movingState == currentState):
                        currentStateAlreadyOnPath = True
                        break
                
                # Only put the current state onto the stack if it isn't already on the path here
                if currentStateAlreadyOnPath == False:
                    self.stack.append(c)

            # Increment the state count
            count += 1
            
            # Every thousand states print the details
            if(count % 1000 == 0):
                print("------------------")
                currentState.show(True, False, False)
                print("States searched " + str(count))
                print("Tree depth: " + str(currentState.treeDepth))                
                print("STACK " + str(len(self.stack)))
                
        # If we couldn't find a solution reutrn None
        return None
    

Test out the search.

In [ ]:
searcher = DepthFirstSearch()

# Do a search
startBoard = Puzzle8()
startBoard.shuffle()
start = SearchState(startBoard)
goal = SearchState(Puzzle8())
#start = SearchState(Puzzle8())
#goal = SearchState(Puzzle8([["D", "A", "C"], ["F", "B", Puzzle8.blankChar], ["G", "H", "E"]]))

print("Start")
start.show()
print("Goal")
goal.show()

moves = searcher.search(start, goal)

# Print the path to a solution
if moves is not None:
    start.board.printTiles()
    count = 1
    for m in moves:
        print(" ")
        print("Move " + str(count) + " " + m)
        start.board.moveBlank(m)
        start.board.printTiles()
        count += 1

Try both searches for the same test

In [ ]:
# Create a start and a goal state
startBoard = Puzzle8()
startBoard.shuffle()
start = SearchState(startBoard)
goal = SearchState(Puzzle8())

print(" ")
print("BREADTH FIRST SEARCH")
print(" ")

# do a breadth first Search
searcher = BreadthFirstSearch()
startCopy = copy.deepcopy(start)
goalCopy = copy.deepcopy(goal)

print("Start")
startCopy.show()
print("Goal")
goalCopy.show()

moves = searcher.search(startCopy, goalCopy)

# Print the path to a solution
if moves is not None:
    startCopy.board.printTiles()
    count = 1
    for m in moves:
        print(" ")
        print("Move " + str(count) + " " + m)
        startCopy.board.moveBlank(m)
        startCopy.board.printTiles()
        count += 1
        

print(" ")
print("DEPTH FIRST SEARCH")
print(" ")

# do a Depth first Search
searcher = DepthFirstSearch()
startCopy = copy.deepcopy(start)
goalCopy = copy.deepcopy(goal)

print("Start")
startCopy.show()
print("Goal")
goalCopy.show()

moves = searcher.search(startCopy, goalCopy)

# Print the path to a solution
if moves is not None:
    startCopy.board.printTiles()
    count = 1
    for m in moves:
        print(" ")
        print("Move " + str(count) + " " + m)
        startCopy.board.moveBlank(m)
        startCopy.board.printTiles()
        count += 1

------------------
['D', 'H', ' '] | ['G', 'F', 'A'] | ['C', 'E', 'B']
States searched 3000
Tree depth: 1886
STACK 2407
------------------
['C', ' ', 'H'] | ['A', 'E', 'D'] | ['B', 'G', 'F']
States searched 4000
Tree depth: 2515
STACK 3204
------------------
['D', 'A', 'H'] | ['G', 'E', ' '] | ['C', 'F', 'B']
States searched 5000
Tree depth: 3141
STACK 4003
